# Surprise homework

In [113]:
import io 
from collections import defaultdict
import pandas as pd
import numpy as np
from surprise import Dataset
from surprise import SVD
from surprise import KNNBasic
from surprise import NormalPredictor
from surprise import KNNWithMeans
from surprise import accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import KFold
from surprise.model_selection import train_test_split
from statistics import mean

### Functions

In [56]:
# используйте полезные функции из FAQ
def precision_recall_at_k(predictions, k=5, threshold=3.52):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls


def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n



### Load data

In [2]:
data = Dataset.load_builtin('ml-100k')
data

In [3]:
trainset, testset = train_test_split(data, test_size=.25)
trainset

In [13]:
ratings = trainset.all_ratings()
print(testset)

[('940', '150', 3.0), ('298', '1', 5.0), ('592', '323', 1.0), ('213', '214', 5.0), ('889', '427', 4.0), ('932', '189', 5.0), ('567', '59', 5.0), ('540', '1', 3.0), ('668', '896', 4.0), ('344', '619', 4.0), ('270', '70', 5.0), ('450', '277', 3.0), ('70', '751', 4.0), ('431', '303', 4.0), ('524', '607', 3.0), ('279', '660', 4.0), ('927', '8', 4.0), ('76', '100', 5.0), ('655', '692', 3.0), ('867', '318', 5.0), ('197', '510', 5.0), ('567', '50', 1.0), ('787', '300', 4.0), ('794', '285', 5.0), ('442', '230', 3.0), ('158', '154', 4.0), ('887', '9', 2.0), ('430', '151', 4.0), ('230', '203', 2.0), ('502', '890', 2.0), ('334', '175', 4.0), ('146', '311', 4.0), ('929', '483', 4.0), ('669', '649', 4.0), ('634', '281', 4.0), ('654', '678', 4.0), ('878', '155', 3.0), ('305', '529', 5.0), ('927', '94', 2.0), ('783', '301', 4.0), ('496', '33', 4.0), ('548', '751', 4.0), ('532', '601', 3.0), ('566', '392', 4.0), ('732', '690', 5.0), ('442', '800', 3.0), ('887', '90', 5.0), ('612', '117', 4.0), ('615',

### Define algorithms

In [ ]:
# внимательно изучите документацию по метрикам и алгоритмам


### Select algorithm

In [28]:
#прогнозирование случайного рейтинга на основе распределения всех рейтингов в наборе
algo = NormalPredictor()
algo.fit(trainset)
predictions = algo.test(testset)
rmse = accuracy.rmse(predictions)
cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=True)
print("NormalPredictor")

RMSE: 1.5158
Evaluating RMSE of algorithm NormalPredictor on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.5122  1.5192  1.5172  1.5228  1.5125  1.5168  0.0040  
Fit time          0.10    0.12    0.12    0.12    0.12    0.11    0.01    
Test time         0.14    0.10    0.10    0.15    0.10    0.12    0.02    
NormalPredictor


In [35]:
sim_options = {'name': 'cosine' }
algo = KNNWithMeans(k=30,sim_options=sim_options)
algo.fit(trainset)
predictions = algo.test(testset)
rmse = accuracy.rmse(predictions)
cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=True)
print("KNNWithMeans cosine")

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9575
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9626  0.9582  0.9676  0.9476  0.9543  0.9580  0.0069  
Fit time          1.44    1.46    1.46    1.49    1.44    1.46    0.02    
Test time         2.78    2.71    2.70    2.80    2.72    2.74    0.04    
KNNWithMeans cosine


In [36]:
sim_options = {'name': 'msd' }
algo = KNNWithMeans(k=30,sim_options=sim_options)
algo.fit(trainset)
predictions = algo.test(testset)
rmse = accuracy.rmse(predictions)
cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=True)
print("KNNWithMeans msd")

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9516
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9587  0.9491  0.9529  0.9486  0.9499  0.9518  0.0037  
Fit time          0.43    0.50    0.48    0.44    0.43    0.46    0.03    
Test time         2.85    2.98    2.83    2.70    2.56    2.79    0.14    
KNNWithMeans msd


In [37]:
sim_options = {'name': 'pearson' }
algo = KNNWithMeans(k=30,sim_options=sim_options)
algo.fit(trainset)
predictions = algo.test(testset)
rmse = accuracy.rmse(predictions)
cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=True)
print("KNNWithMeans pearson")

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.9519
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9585  0.9569  0.9471  0.9459  0.9480  0.9513  0.0053  
Fit time          1.78    1.81    1.79    1.86    1.84    1.82    0.03    
Test time         2.73    2.65    2.60    2.63    2.75    2.67    0.06    
KNNWithMeans pearson


In [32]:
#SVD алгоритм
algo = SVD()
algo.fit(trainset)
predictions = algo.test(testset)
rmse = accuracy.rmse(predictions)
cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=True)
print("SVD")

RMSE: 0.9368
Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9314  0.9450  0.9314  0.9381  0.9398  0.9372  0.0052  
Fit time          3.76    3.86    3.82    3.82    3.86    3.83    0.04    
Test time         0.11    0.15    0.11    0.11    0.20    0.13    0.04    
SVD


### Calculate precision@k and recall@k

In [55]:
algo = SVD()
kf = KFold(n_splits=5)
meanPrecis=[]
meanRacall=[]
for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=3.52)

    # Precision and recall can then be averaged over all users
    meanPrecis.append(sum(prec for prec in precisions.values()) / len(precisions))
    meanRacall.append(sum(rec for rec in recalls.values()) / len(recalls))

print("precision@k")
print(mean(meanPrecis))
print("recall@k")
print(mean(meanRacall))

precision@k
0.7223916043361983
recall@k
0.40356582649688666


In [52]:
ds=[0.40443128029284925,
 0.4014990022234645,
 0.4091604950720822,
 0.4103923500744347,
 0.4068611673507407]
print(mean(ds))

0.4064688590027143


### Predict

In [120]:
# обратите внимание на функцию build_anti_testset
# First train an SVD algorithm on the movielens dataset.
user='9'
data = Dataset.load_builtin('ml-100k')
trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)
item_path = 'C:/Users/МрКонтик/.surprise_data/ml-100k/u.item'
item_df = pd.read_csv(item_path, sep='|',encoding='ISO-8859-1', header = None) 
# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=5)
print("User "+user)
for filmRating in top_n[user]:
    filmIndex=int(filmRating[0])-1
    print("  "+str(filmRating[0]) + " ('" +item_df[filmIndex:filmIndex+1][1].values[0]+"', '"+item_df[filmIndex:filmIndex+1][2].values[0]+"') " + str(np.around(filmRating[1],decimals = 3)))

User 9
  64 ('Shawshank Redemption, The (1994)', '01-Jan-1994') 5
  169 ('Wrong Trousers, The (1993)', '01-Jan-1993') 5
  114 ('Wallace & Gromit: The Best of Aardman Animation (1996)', '05-Apr-1996') 5
  98 ('Silence of the Lambs, The (1991)', '01-Jan-1991') 4.982
  178 ('12 Angry Men (1957)', '01-Jan-1957') 4.968


In [121]:
print("User "+user)
for filmRating in top_n[user]:
    filmIndex=int(filmRating[0])-1
    print("  "+str(filmRating[0]) + " ('" +item_df[filmIndex:filmIndex+1][1].values[0]+"', '"+item_df[filmIndex:filmIndex+1][2].values[0]+"') " + str(np.around(filmRating[1],decimals = 3)))

User 9
  64 ('Shawshank Redemption, The (1994)', '01-Jan-1994') 5
  169 ('Wrong Trousers, The (1993)', '01-Jan-1993') 5
  114 ('Wallace & Gromit: The Best of Aardman Animation (1996)', '05-Apr-1996') 5
  98 ('Silence of the Lambs, The (1991)', '01-Jan-1991') 4.982
  178 ('12 Angry Men (1957)', '01-Jan-1957') 4.968


In [90]:
item_path = 'C:/Users/МрКонтик/.surprise_data/ml-100k/u.item'
item_df = pd.read_csv(item_path, sep='|',encoding='ISO-8859-1', header = None)
print(item_df[1239:1240][1].values[0])

Ghost in the Shell (Kokaku kidotai) (1995)
